In [ ]:
!pip install eli5==0.13.0

# 한글 폰트 사용을 위한 라이브러리입니다.
!apt-get install -y fonts-nanum

In [9]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')
import os

In [13]:
folder = "/data/ephemeral/home/upstageailab-ml-competition-ml-6/sohyun/FastFood"
files = [
    "서울시 일반음식점 인허가 정보_롯데리아.xlsx",
    "서울시 일반음식점 인허가 정보_맘스터치.xlsx",
    "서울시 일반음식점 인허가 정보_맥도날드.xlsx",
    "서울시 일반음식점 인허가 정보_버거킹.xlsx",
    "서울시 일반음식점 인허가 정보_케이에프씨.xlsx",
    "서울시 휴게음식점 인허가 정보_롯데리아.xlsx",
    "서울시 휴게음식점 인허가 정보_맘스터치.xlsx",
    "서울시 휴게음식점 인허가 정보_맥도날드.xlsx",
    "서울시 휴게음식점 인허가 정보_버거킹.xlsx",
    "서울시 휴게음식점 인허가 정보_케이에프씨.xlsx"
]

df_list = [pd.read_excel(os.path.join(folder, f), engine="openpyxl") for f in files]
merged_df = pd.concat(df_list, ignore_index=True)

print(merged_df.shape)
print(merged_df.columns)

(1465, 44)
Index(['개방자치단체코드', '관리번호', '인허가일자', '인허가취소일자', '영업상태코드', '영업상태명', '상세영업상태코드',
       '상세영업상태명', '폐업일자', '휴업시작일자', '휴업종료일자', '재개업일자', '전화번호', '소재지면적',
       '소재지우편번호', '지번주소', '도로명주소', '도로명우편번호', '사업장명', '최종수정일자', '데이터갱신구분',
       '데이터갱신일자', '업태구분명', '좌표정보(X)', '좌표정보(Y)', '위생업태명', '남성종사자수', '여성종사자수',
       '영업장주변구분명', '등급구분명', '급수시설구분명', '총인원', '본사종업원수', '공장사무직종업원수',
       '공장판매직종업원수', '공장생산직종업원수', '건물소유구분명', '보증액', '월세액', '다중이용업소여부', '시설총규모',
       '전통업소지정번호', '전통업소주된음식', '홈페이지'],
      dtype='object')


In [23]:
#'영업/정상' 상태의 데이터 추출 및 저장
merged_df = merged_df[merged_df['영업상태명'] == '영업/정상']

#필요로 하는 컬럼만 포함하는 새로운 데이터프레임 생성
selected_columns = ['도로명주소', '도로명우편번호', '사업장명', '좌표정보(X)', '좌표정보(Y)', '영업장주변구분명', '시설총규모']
selected_df = merged_df[selected_columns]

print(selected_df.head())

                                  도로명주소  도로명우편번호              사업장명  \
0         서울특별시 중구 남대문로 30 (남창동,1층143호)   4529.0          롯데리아남대문점   
6   서울특별시 종로구 청계천로 41, 영풍빌딩 지하 2층 (서린동)   3188.0  롯데리아 영풍문고 종각종로본점   
10              서울특별시 광진구 면목로 136 (중곡동)   4908.0           롯데리아중곡점   
11      서울특별시 성북구 동소문로 2, 1,2층 (동소문동2가)   2860.0     롯데리아(한성대입구역점)   
13   서울특별시 노원구 한글비석로 369, 롯데리아 1층 (중계동)   1699.0          롯데리아 상계점   

          좌표정보(X)        좌표정보(Y) 영업장주변구분명   시설총규모  
0   198095.795196  450940.108370       기타  114.38  
6   198351.818737  451921.603026      NaN  191.74  
10  207102.532983  451463.082422       기타   99.34  
11  200528.744139  454019.178903      NaN  238.30  
13  206460.340838  461782.719034      NaN  165.00  


In [27]:
#브랜드 값을 추출하여 새로운 컬럼에 저장
def extract_brand(name):
    for brand in ['롯데리아', '맘스터치', '맥도날드', '케이에프씨', '버거킹']:
        if brand in name:
            return brand
    return '기타'  # 혹시 모를 예외 처리

selected_df['브랜드'] = selected_df['사업장명'].apply(extract_brand)

print(selected_df.head())
print(selected_df['브랜드'].value_counts())

                                  도로명주소  도로명우편번호              사업장명  \
0         서울특별시 중구 남대문로 30 (남창동,1층143호)   4529.0          롯데리아남대문점   
6   서울특별시 종로구 청계천로 41, 영풍빌딩 지하 2층 (서린동)   3188.0  롯데리아 영풍문고 종각종로본점   
10              서울특별시 광진구 면목로 136 (중곡동)   4908.0           롯데리아중곡점   
11      서울특별시 성북구 동소문로 2, 1,2층 (동소문동2가)   2860.0     롯데리아(한성대입구역점)   
13   서울특별시 노원구 한글비석로 369, 롯데리아 1층 (중계동)   1699.0          롯데리아 상계점   

          좌표정보(X)        좌표정보(Y) 영업장주변구분명   시설총규모   브랜드  
0   198095.795196  450940.108370       기타  114.38  롯데리아  
6   198351.818737  451921.603026      NaN  191.74  롯데리아  
10  207102.532983  451463.082422       기타   99.34  롯데리아  
11  200528.744139  454019.178903      NaN  238.30  롯데리아  
13  206460.340838  461782.719034      NaN  165.00  롯데리아  
맘스터치     189
롯데리아     173
버거킹      107
맥도날드      86
케이에프씨     38
Name: 브랜드, dtype: int64


In [ ]:
import requests
import time
import csv

# 1) 내 네이버 API 키
CLIENT_ID     = 'YOUR_CLIENT_ID'
CLIENT_SECRET = 'YOUR_CLIENT_SECRET'

# 2) 검색 대상 브랜드 리스트
brands = ['KFC', '맥도날드', '맘스터치', '롯데리아', '버거킹']

# 3) API 엔드포인트
URL = 'https://naveropenapi.apigw.ntruss.com/map-place/v1/search'

# 4) 헤더 설정
headers = {
    'X-NCP-APIGW-API-KEY-ID':     ncp_iam_BPAMKR4Is1SWL5u5QCAc,
    'X-NCP-APIGW-API-KEY':        ncp_iam_BPKMKR6fETnuLaj58qxsCXr9dkeaK9zAMe
}

# 5) 결과를 담을 리스트
all_locations = []

# 6) 브랜드별로 페이지(1~5) 순환하며 수집
for brand in brands:
    for page in range(1, 6):
        params = {
            'query': f'{brand} 서울',
            'page':  page,
            'size':  15   # 한 페이지에 최대 15건까지
        }
        resp = requests.get(URL, headers=headers, params=params)
        resp.raise_for_status()
        data = resp.json().get('places', [])
        if not data:
            break  # 더 이상 결과가 없으면 다음 브랜드로

        for place in data:
            all_locations.append({
                'brand':   brand,
                'name':    place.get('name'),
                'address': place.get('road_address') or place.get('jibun_address'),
                'lng':     place['x'],
                'lat':     place['y']
            })
        time.sleep(0.1)  # 초당 호출 제한 준수

# 7) CSV로 저장
with open('burger_locations.csv', 'w', newline='', encoding='utf-8-sig') as f:
    writer = csv.DictWriter(f, fieldnames=['brand','name','address','lng','lat'])
    writer.writeheader()
    writer.writerows(all_locations)

print(f'수집 완료: 총 {len(all_locations)}개 지점 저장됨.')
